# <font color='blue'> 1. PointNet Object Classification using Deep Learning  </font> 
- Introduction 
- Network Architecture
- Hands-On Experience

## 1.1 Introduction
Point Cloud is an important geometrical datatype that is canonical (depth, lidar) but irregular. Due to its irregularity, most research works focus on transforming the point cloud data to regular 3D voxel grids (3D-CNN) or collections of images (2D-CNN). However, such transformation leads to various issues, and it becomes voluminous. The point cloud transformation may lead to losing the basic structure (or features) of point cloud data. </font> 

<font color = 'black'> To end this, PointNet was proposed in 2017 that focuses on learning a model on raw point cloud data. The network is the first one in this area, and basic but it is robust to perturbation and corruption. It is efficient and effective in many point cloud tasks such as object classification, part segmentation and semantic segmentation. 

In the following, we will see object classification with this method using the ModelNet40 dataset.  It is possible to extend this method it to any custom dataset.

For further details, the article is available at: https://arxiv.org/abs/1612.00593 

Reference: Qi, Charles R., et al. "Pointnet: Deep learning on point sets for 3d classification and segmentation." Proceedings of the IEEE conference on computer vision and pattern recognition. 2017. </font> 

## 1.2 Network Architecture: PointNet
<img src="../../description/pointnet.png">

## 1.3 Hands-On-Experience

Import Required Modules

In [ ]:
import os
import torch
import numpy as np
from tqdm import tqdm
from dataloaders.ModelNetDataLoader import ModelNetDataLoader
from utilities.data_provide import random_point_dropout, random_scale_point_cloud, shift_point_cloud

<font color = 'red'> Parameters </font>

In [ ]:
class Args:
    '''PARAMETERS'''
    use_cpu =False
    gpu='0'
    batch_size = 24
    model='pointnet_cls'
    num_category = 40
    epoch=200
    learning_rate=0.001
    num_point=1024
    optimizer='Adam'
    log_dir = 'runs'
    decay_rate=1e-4
    use_normals=False
    process_data=False
    use_uniform_sample=False

args = Args()

Test Function for the test dataloader. Used inside the training loop.

In [ ]:

def test(model, loader, num_class=40):
    mean_correct = []
    class_acc = np.zeros((num_class, 3))
    classifier = model.eval()

    for j, (points, target) in tqdm(enumerate(loader), total=len(loader)):

        if not args.use_cpu:
            points, target = points.cuda(), target.cuda()

        points = points.transpose(2, 1)
        pred, _ = classifier(points)
        pred_choice = pred.data.max(1)[1]

        for cat in np.unique(target.cpu()):
            classacc = pred_choice[target == cat].eq(target[target == cat].long().data).cpu().sum()
            class_acc[cat, 0] += classacc.item() / float(points[target == cat].size()[0])
            class_acc[cat, 1] += 1

        correct = pred_choice.eq(target.long().data).cpu().sum()
        mean_correct.append(correct.item() / float(points.size()[0]))

    class_acc[:, 2] = class_acc[:, 0] / class_acc[:, 1]
    class_acc = np.mean(class_acc[:, 2])
    instance_acc = np.mean(mean_correct)

    return instance_acc, class_acc

Inplace Relu saves memory!!!

In [ ]:
def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace=True

We do require a GPU. Define the GPU value in a cluster of GPUs. By default it is 0 (a single GPU environment).

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

### Dataloader Part: 
We choose ModelNet_40 dataset for our implementation. It is a classification dataset that consists of 40 classes for household objects. The training and testing split is defined inside the modelnet40_normal_resampled directory. The data represents CAD models which are cleaned inhouse.

For our implementation, we are using the model resampled version (Aligned).

The aligned dataset is provided by N. Sedaghat, M. Zolfaghari, E. Amiri and T. Brox, the authors of Orientation-boosted Voxel Nets for 3D Object Recognition [8].

The CAD models are in Object File Format (OFF). Matlab functions are provide with the dataset to read and visualize OFF files in Princeton Vision Toolkit (PVT).

The dataset and the related information is available at: https://modelnet.cs.princeton.edu

You can also use ModelNet10 dataset. This dataset consists of 10 classes instead of 40 classes.


#### <font color = 'red'> How to download the dataset and prepare it accordingly? </font>

To download the dataset and prepare it for the training, follow the given instruction. 

1. Go the the link to download the data
https://modelnet.cs.princeton.edu

2. Unzip the data and place the unzipped directory inside data directory.
(Note: the 'data' directory is already created for you. You do not need to create it again. You need to place the unzipped folder inside this directory only.)

Note: The data directory name should be checked. It is 'modelnet40_normal_resampled' by default. If it is changed, do check the DATA_PATH in the following block.

In [ ]:
data_path = '../../data/modelnet40_normal_resampled/'

train_dataset = ModelNetDataLoader(root=data_path, args=args,  split='train', process_data=args.process_data)
test_dataset = ModelNetDataLoader(root=data_path, args=args, split='test', process_data=args.process_data)
trainDataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=10, drop_last=True)
testDataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=10)


### Model Part: 

In [ ]:
'''MODEL LOADING'''
num_class = args.num_category
from models.pointnet_cls import get_model, get_loss
classifier = get_model(num_class, normal_channel=args.use_normals)
criterion = get_loss()
classifier.apply(inplace_relu)

Weight save directory and Optimizer

In [ ]:
if not args.use_cpu:
    classifier = classifier.cuda()
    criterion = criterion.cuda()

try:
    checkpoint = torch.load('weights/best_model_classification.pth')
    start_epoch = checkpoint['epoch']
    classifier.load_state_dict(checkpoint['model_state_dict'])
    print('Use pretrain model')
except:
    print('No existing model, starting training from scratch...')
    start_epoch = 0

if args.optimizer == 'Adam':
    optimizer = torch.optim.Adam(
        classifier.parameters(),
        lr=args.learning_rate,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=args.decay_rate
    )
else:
    optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)

Schedular and Model Initialization

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.7)
global_epoch = 0
global_step = 0
best_instance_acc = 0.0
best_class_acc = 0.0

In [ ]:
print('Start training...')
for epoch in range(start_epoch, args.epoch):
    print('Epoch %d (%d/%s):' % (global_epoch + 1, epoch + 1, args.epoch))
    mean_correct = []
    classifier = classifier.train()

    scheduler.step()
    for batch_id, (points, target) in tqdm(enumerate(trainDataLoader, 0), total=len(trainDataLoader), smoothing=0.9):
        optimizer.zero_grad()

        points = points.data.numpy()
        points = random_point_dropout(points)
        points[:, :, 0:3] = random_scale_point_cloud(points[:, :, 0:3])
        points[:, :, 0:3] = shift_point_cloud(points[:, :, 0:3])
        points = torch.Tensor(points)
        points = points.transpose(2, 1)

        if not args.use_cpu:
            points, target = points.cuda(), target.cuda()

        pred, trans_feat = classifier(points)
        loss = criterion(pred, target.long(), trans_feat)
        pred_choice = pred.data.max(1)[1]

        correct = pred_choice.eq(target.long().data).cpu().sum()
        mean_correct.append(correct.item() / float(points.size()[0]))
        loss.backward()
        optimizer.step()
        global_step += 1

    train_instance_acc = np.mean(mean_correct)
    print('Train Instance Accuracy: %f' % train_instance_acc)

    with torch.no_grad():
        instance_acc, class_acc = test(classifier.eval(), testDataLoader, num_class=num_class)

        if (instance_acc >= best_instance_acc):
            best_instance_acc = instance_acc
            best_epoch = epoch + 1

        if (class_acc >= best_class_acc):
            best_class_acc = class_acc
        print('Test Instance Accuracy: %f, Class Accuracy: %f' % (instance_acc, class_acc))
        print('Best Instance Accuracy: %f, Class Accuracy: %f' % (best_instance_acc, best_class_acc))

        if (instance_acc >= best_instance_acc):
            print('Save model...')
            savepath = 'weights/' + 'best_model_classification.pth'
            print('Saving at %s' % savepath)
            state = {
                'epoch': best_epoch,
                'instance_acc': instance_acc,
                'class_acc': class_acc,
                'model_state_dict': classifier.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }
            torch.save(state, savepath)
        global_epoch += 1

print('End of training...')
